In [12]:
import pandas as pd
import numpy as np
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [13]:
def rank(subtitle_file,excel_name,sheet_name):
    
    #read input file
    file = open(subtitle_file)
    SENTENCES_COUNT =0         
    data = file.read()

    CoList = data.split('\n')

    for i in CoList:
        if i:        
            SENTENCES_COUNT += 1

    data = data.replace('\n', ' . ')
    #close the input file
    file.close()

    ############
    #open the input file in write mode
    fin = open(subtitle_file, "wt")
    #overrite the input file with the resulting data
    fin.write(data)
    #close the file
    fin.close()

    ###### read again####
    document = open(subtitle_file).read()

    ####PANDAS####
    df = pd.read_excel(excel_name,header=0,sheet_name=sheet_name,index_col=0)
    df.index.name = None
    df =df.fillna(0)
    df= df.astype(int)
    

    #####PYSUMMM#####
    
    auto_abstractor = AutoAbstractor() # Object of automatic summarization.
    auto_abstractor.tokenizable_doc = SimpleTokenizer() # Set tokenizer.
    auto_abstractor.delimiter_list = ["."] # Set delimiter for making a list of sentence.
    abstractable_doc = TopNRankAbstractor()# Object of abstracting and filtering document.
    
    for n in range(1,SENTENCES_COUNT+1):
        abstractable_doc.set_top_n(n) # set n = 120 value
        result_dict = auto_abstractor.summarize(document, abstractable_doc) # Summarize document.
        #print(n )

        for i in range(n):
            a=result_dict["scoring_data"][i][0]+1
            #print('a ='+str(a))
            if df['pysum'][a]==0:
                df['pysum'][a] = n

    #####SUMY#########
    LANGUAGE = "english"
    parser = PlaintextParser.from_file(subtitle_file, Tokenizer(LANGUAGE))
    summarizer_lex = LexRankSummarizer()

    summarizer_luhn = LuhnSummarizer()

    summarizer_lsa = LsaSummarizer()

    summarizer_textrank = TextRankSummarizer()

    for n in range(1,SENTENCES_COUNT):
        summary_lex = summarizer_lex(parser.document,n)
        summary_luhn = summarizer_luhn(parser.document,n)
        summary_lsa = summarizer_lsa(parser.document,n)
        summary_textrank = summarizer_textrank(parser.document,n)
        for sentence in summary_lex:        
            a=parser.document.sentences.index(sentence)+1
            #print('a ='+str(a))
            if df['lexrank'][a]==0:
                df['lexrank'][a] = n
        for sentence in summary_luhn:        
            a=parser.document.sentences.index(sentence)+1
            #print('a ='+str(a))
            if df['luhn'][a]==0:
                df['luhn'][a] = n
        for sentence in summary_lsa:        
            a=parser.document.sentences.index(sentence)+1
            #print('a ='+str(a))
            if df['lsa'][a]==0:
                df['lsa'][a] = n
        for sentence in summary_textrank:        
            a=parser.document.sentences.index(sentence)+1
            #print('a ='+str(a))
            if df['textrank'][a]==0:
                df['textrank'][a] = n

    return df

In [14]:
rank(subtitle_file='Sub_1.txt',excel_name='data.xlsx',sheet_name='Sheet1')

,pysum,lexrank,luhn,lsa,textrank,x1.,x1.3,x1.6,x1.9,x2.2,x2.5,nadir_kelime,konu_ile_alakalı
1,8,78,63,109,88,1,1,1,1,1,1,0,0
2,11,64,102,24,83,1,1,1,1,1,1,0,0
3,10,71,83,26,106,1,1,1,1,1,0,0,0
4,7,110,7,19,12,1,1,1,1,1,1,0,0
5,6,2,3,16,4,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,99,84,97,115,0,1,1,1,1,1,1,0,0
117,49,25,21,21,43,1,1,1,1,1,1,0,0
118,39,31,32,14,16,1,1,1,1,1,0,0,0
119,24,38,33,1,1,1,1,1,1,1,0,0,0
